## Preprocessing

In [301]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [302]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"], axis=1, inplace=True)

In [303]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [304]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type = application_df["APPLICATION_TYPE"].value_counts()
app_type 

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [305]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# all application types with a count lower than 1000 will be treated is outliers and will be rolled up into other
application_types_to_replace = list(app_type[app_type < 1000].keys())

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [306]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [307]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification = application_df["CLASSIFICATION"].value_counts()
classification[classification > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [308]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# all classifications with a count lower than 2000 will be treated is outliers and will be rolled up into other
classifications_to_replace = list(classification[classification < 2000].keys())

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [309]:
# Convert categorical data to numeric with `pd.get_dummies`
cate_columns = list(application_df.dtypes[application_df.dtypes == object].keys())
application_dummies = pd.get_dummies(application_df, columns=cate_columns, dtype=int)

In [310]:
# Split our preprocessed data into our features and target arrays
y = application_dummies["IS_SUCCESSFUL"]
X = application_dummies.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [311]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [312]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\cadkm\anaconda3\envs\inclassfeb2024\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_90 (Dense)                │ (None, 120)            │         4,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 60)             │         7,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_92 (Dense)                │ (None, 30)             │         1,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ (None, 15)             │           465 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,251 (55.67 KB)

 Trainable params: 14,251 (55.67 KB)

 Non-trainable params: 0 (0.00 B)

In [313]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [314]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 674us/step - accuracy: 0.7077 - loss: 0.5930
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 664us/step - accuracy: 0.7238 - loss: 0.5629
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 653us/step - accuracy: 0.7333 - loss: 0.5554
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 673us/step - accuracy: 0.7254 - loss: 0.5624
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 748us/step - accuracy: 0.7272 - loss: 0.5605
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.7293 - loss: 0.5568
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step - accuracy: 0.7319 - loss: 0.5512
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step - accuracy: 0.7282 - loss: 0.5578
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 668us/step - accuracy: 0.7290 - loss: 0.5571
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - accuracy: 0.7309 - loss: 0.5555
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step - accuracy: 0.7276 - loss: 0.5576
Epoch 12/50
804/804 ━━━━━━━━━━

In [315]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 870us/step - accuracy: 0.7252 - loss: 0.5637
Loss: 0.5637490749359131, Accuracy: 0.7252478003501892


In [316]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")